<a href="https://colab.research.google.com/github/Muppidathe/deep_learning/blob/main/lenet_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torchvision
from torchvision import transforms

In [2]:
transformer=transforms.Compose([transforms.Resize((28,28)),transforms.ToTensor(),transforms.Normalize(mean=0.1307,std=.3081)])
train_dataset=torchvision.datasets.MNIST(root='./data',train=True,transform=transformer,download=True)
test_dataset=torchvision.datasets.MNIST(root='./data',train=False,transform=transformer,download=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 60.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.71MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 15.0MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.36MB/s]


In [3]:
from torch.utils.data import DataLoader
train_data_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_data_loader=DataLoader(test_dataset,batch_size=32,shuffle=True)

#formula
(Input + 2Padding -(kernal_size - 1) - 1) / stride + 1

In [4]:
import torch.nn as nn
class lenet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1=nn.Conv2d(1,6,kernel_size=5,stride=1,padding=2)#(28*28/1 -> 28*28/6)
    self.relu=nn.ReLU()
    self.pool1=nn.AvgPool2d(kernel_size=2,stride=2)#(28*28/6 -> 14*14/6)
    self.conv2=nn.Conv2d(6,16,kernel_size=5,stride=1)#(14*14/6 -> 10*10/16)
    self.pool2=nn.AvgPool2d(kernel_size=2,stride=2)#(10*10/16 -.5*5/16)
    self.flatten=nn.Flatten()
    self.dense1=nn.Linear(16*5*5,120)
    self.dense2=nn.Linear(120,84)
    self.dense3=nn.Linear(84,10)
  def forward(self,x):
    x=self.pool1(self.relu(self.conv1(x)))
    x=self.pool2(self.relu(self.conv2(x)))
    x=self.flatten(x)
    x=self.relu(self.dense1(x))
    x=self.relu(self.dense2(x))
    x=self.dense3(x)
    x=nn.functional.softmax(x,dim=1)
    return x

In [9]:
import torch
from torch import optim
model=lenet()
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer=optim.Adam(model.parameters(),lr=.001)
criterion=nn.CrossEntropyLoss()
epochs=10
for epoch in range(epochs):
  running_loss=[]
  total,correct=0,0
  model.train()
  for image,label in train_data_loader:
    image=image.to(device)
    label=label.to(device)
    output=model(image)
    optimizer.zero_grad()
    loss=criterion(output,label)
    loss.backward()
    optimizer.step()
    _,predicted=torch.max(output,1)
    total+=label.size(0)
    correct+=(predicted==label).sum().item()
    running_loss.append(loss.item())
  train_avg_loss=sum(running_loss)/len(running_loss)
  train_accuracy=100*correct/total
  model.eval()
  running_loss=[]
  total,correct=0,0
  with torch.no_grad():
    image=image.to(device)
    label=label.to(device)
    for image,label in test_data_loader:
      image=image.to(device)
      label=label.to(device)
      output=model(image)
      loss=criterion(output,label)
      running_loss.append(loss.item())
      _,predicted=torch.max(output,1)
      total+=label.size(0)
      correct+=(predicted==label).sum().item()
  accuracy=100*correct/total
  avg_loss=sum(running_loss)/len(running_loss)
  print(f'Epoch [{epoch+1}/{epochs}]--> train loss {train_avg_loss} train accuracy {train_accuracy} test loss {avg_loss} test accuracy {accuracy}')



Epoch [1/10]--> train loss 1.5695996491750082 train accuracy 89.55666666666667 test loss 1.4991713529958512 test accuracy 96.32
Epoch [2/10]--> train loss 1.4932534084320068 train accuracy 96.84833333333333 test loss 1.4877289655490424 test accuracy 97.47
Epoch [3/10]--> train loss 1.4869401298522948 train accuracy 97.45833333333333 test loss 1.4875239644187708 test accuracy 97.37
Epoch [4/10]--> train loss 1.483885770924886 train accuracy 97.74333333333334 test loss 1.4783154299464851 test accuracy 98.31
Epoch [5/10]--> train loss 1.480341119893392 train accuracy 98.08833333333334 test loss 1.47846042233915 test accuracy 98.28
Epoch [6/10]--> train loss 1.479487887763977 train accuracy 98.185 test loss 1.4782655022014826 test accuracy 98.26
Epoch [7/10]--> train loss 1.4777521405537923 train accuracy 98.34333333333333 test loss 1.4779594005487215 test accuracy 98.32
Epoch [8/10]--> train loss 1.4766102633158367 train accuracy 98.45833333333333 test loss 1.4745299663787452 test accurac